In [1]:
import pandas as pd

In [84]:
# reading data in csv file using pandas
df = pd.read_csv('training.1600000.processed.noemoticon.csv', encoding="ISO-8859-1", header=None, names=['target', 'id', 'date', 'flag', 'user', 'text',])

In [3]:
df.head()

,target,id,date,flag,user,text
0,0,1467810369,Mon Apr 06 22:19:45 PDT 2009,NO_QUERY,_TheSpecialOne_,"@switchfoot http://twitpic.com/2y1zl - Awww, t..."
1,0,1467810672,Mon Apr 06 22:19:49 PDT 2009,NO_QUERY,scotthamilton,is upset that he can't update his Facebook by ...
2,0,1467810917,Mon Apr 06 22:19:53 PDT 2009,NO_QUERY,mattycus,@Kenichan I dived many times for the ball. Man...
3,0,1467811184,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,ElleCTF,my whole body feels itchy and like its on fire
4,0,1467811193,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,Karoli,"@nationwideclass no, it's not behaving at all...."


In [64]:
# Finding out the top tweeters 
number_tweets_df = df.groupby('user').text.count().reset_index()
number_tweets_df.columns = ['user', 'number of tweets']

number_tweets_df.sort_values(by=['number of tweets'], ascending=False, inplace=True)
number_tweets_df.head()

,user,number of tweets
474340,lost_dog,549
641524,webwoke,345
630446,tweetpet,310
195210,SallytheShizzle,281
233109,VioletsCRUK,279


In [65]:
# Number of people with more than _ tweets
number_of_tweets = 20
sum(df.groupby('user').count()['target'] > number_of_tweets)

5725

In [77]:
df_20_tweets = df.groupby('user').count()['target'] > number_of_tweets
df_20_tweets = df_20_tweets.reset_index()
tweeters_with_more_20tweets = df_20_tweets[df_20_tweets['target'] == True]['user'].tolist()
len(tweeters_with_more_20tweets)

5725

In [88]:
df_more_20tweets = df[df['user'].isin(tweeters_with_more_20tweets)]
df_more_20tweets.to_csv('please_work.csv')

TypeError: to_csv() got an unexpected keyword argument 'file_name'

In [87]:
df_more_20tweets.to_csv('please_work.csv')

In [5]:
# Creating lists of users and texts 
users = list(df['user'])
texts = list(df['text'])

In [6]:
# Creating data format to send to IBM Watson
user_dictionary = {}
for i in range(len(users)):
    user = users[i]
    text = texts[i]
    
    tweet = {}
    
    tweet['content'] = text
    tweet['contenttype'] = 'text/plain'
    tweet['language'] = 'en'
    
    if user not in user_dictionary:
        user_dictionary[user] = []
        user_dictionary[user].append(tweet)
    else:
        user_dictionary[user].append(tweet)

In [8]:
# Final step for data to follow IBM Watson's format 
def create_data(list_of_dic):
    data = {}
    data['contentItems'] = list_of_dic
    
    return data

In [9]:
data = create_data(user_dictionary['VioletsCRUK'])

<div class="alert alert-info">
    Getting the IBM Watson thingy to work below
</div>

In [10]:
import json
from ibm_watson import PersonalityInsightsV3
from ibm_cloud_sdk_core.authenticators import IAMAuthenticator

#API KEY provided on the service page 
KEY = 'STximBrq6oOabT5yZteafZcbt3m6Xh6SUSVuWr3RW3V6'

# Authentication via IBM's IAM (Identity and Access Management)
authenticator = IAMAuthenticator(KEY)

# Creating a service instance
service = PersonalityInsightsV3(
    version='2017-10-13',
    authenticator=authenticator)

# Setting service endpoint 
service.set_service_url('https://gateway.watsonplatform.net/personality-insights/api')

# getting result of data sent 
profile = service.profile(
    data,
    'application/json',
    raw_scores=True,
    consumption_preferences=True).get_result()

print(json.dumps(profile, indent=2))

{
  "word_count": 5024,
  "processed_language": "en",
  "personality": [
    {
      "trait_id": "big5_openness",
      "name": "Openness",
      "category": "personality",
      "percentile": 0.16331892999233605,
      "raw_score": 0.7173344937778867,
      "significant": true,
      "children": [
        {
          "trait_id": "facet_adventurousness",
          "name": "Adventurousness",
          "category": "personality",
          "percentile": 0.232163568307652,
          "raw_score": 0.4787924480657912,
          "significant": true
        },
        {
          "trait_id": "facet_artistic_interests",
          "name": "Artistic interests",
          "category": "personality",
          "percentile": 0.40100721757858815,
          "raw_score": 0.6526313944963197,
          "significant": true
        },
        {
          "trait_id": "facet_emotionality",
          "name": "Emotionality",
          "category": "personality",
          "percentile": 0.66235013349263,
         